# 2.3 Example - XANES LCF using Experiment Class

    In this example a first-middle-last scan LCF fit will be compelted and plotted against the known process parameters.

    This example assumes that you have a pickled experiment by previoudly running:
        '2.0 Example - Experiment Class -  XAS Data Processing'
        '2.1 Example - Experiment Class -  Normalized XANES-EXAFS-FT'
        '2.2 Example - Experiment Class -  XAS-Process Stream Correlation'


### Steps:
    
    1. Import Modules
    2. Define Paths
    3. Open Saved Experiment using 'pickle'
    4. Perform LCF
    5. Save Experiment with 'pickle'

### Things to have ready when attemtping this example:

    1. Directory and filename for the 'pickled' experiment 
    2. Scan name of one spectra in the experiment object

## Step 1. Import Modules

In [1]:
# File Handling
import os
import pickle

# Make Plotting interactive
from ipywidgets import interact, fixed

# Wrapper for xraylarch (larch)
os.chdir('..')
pwd = os.getcwd()
os.chdir('./catxas')
import general as fcts
import xas as xfcts
import plot as pfcts
import process
import experiment as exp

# Plot Handling
%matplotlib inline

ModuleNotFoundError: No module named 'larch'

## Step 2. Define Paths

    Note: The sample shown here has mass spec and LabView data.  As more process streams are generated, additional formats can be added. This is just an example.

#### Modify:
    1. pickle_dir - directory containing the pickled experiment
    2. pickpe_name - filename of the pickled experiment (must include extension '.pickle')
    3. scan_name - a name of one fo the spectra in the experiment, used for interrogation throughout

In [ ]:
# Directory Containing Experiment:
pickle_dir = os.path.join(pwd, 'sample results')

# Name of Experiment (must include extension '.pickle')
pickle_name = 'SnO2_TPR.pickle'

# Scan Naame for looking at parts of the Experiment Object
scan_name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001'

## Step 3. Open Saved Experiment using 'pickle'

    Pickle - A method to save the experimental class and reopen it elsewhere.

In [ ]:
pickle_path = os.path.join(pickle_dir,pickle_name)
pickle_in = open(pickle_path,"rb")
my_exp = pickle.load(pickle_in)
pickle_in.close()

## Step 4. Perform LCF

    Notes:
        a. Basis sets must be larch groups that have been calibrated and normalized
        

### Major steps for performing LCF:
    
    a. Generate list of basis spectra groups
    b. Load basis set into a "Fit"
    c. Perform LCF
    d. Generate a Fit Report
    e. Visualize Fit and Errors with one Process Parameter
    f. Visulaize Single Fit
    


### 4a. Generate list of basis spectra groups

    In this section we ae using three basis spectra that are taken from the main data set, not standards.
        1. First Scan (*_0001)
        2. ~Middle Scan (*_0003)
        3. Last Scan (*_0050)
        
    Notes:
        1. Each basis gropu must have its own name.
        2. Basis are compiled into a list prior to uploading into the experiment

In [ ]:
# Basis set made from three spectra in the sample dataset
basis1 = my_exp.spectra['20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001']['Absorption Spectra']['mu Sample'] 
basis1.name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001'

basis2 = my_exp.spectra['20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0031']['Absorption Spectra']['mu Sample']
basis2.name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0031'

basis3 = my_exp.spectra['20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0050']['Absorption Spectra']['mu Sample']
basis3.name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0050'

basis = [basis1, basis2, basis3]

### 4b. Load basis set into a "Fit"

    Each LCF fit can be saved under a different "Fit name". Choosing the same name will overwrite a former Fit

#### MODIFY:
    1. fit_name - unique name for the fit

In [ ]:
# Load list of basis spectra into the experiment
fit_name = 'Fit 1'


my_exp.load_lcf_basis(basis, 'Fit 1')

#### Inspection tool to show how LCF is added to the experiment
    1. Dictionary of fits - experiment.analysis['LCF]
    2. Visualization of the uploaded basis sets

In [ ]:
# 1. Inspect Analysis side of experiment

print('keys in the experiment.analysis object:')
for key in my_exp.analysis.keys():
    print(f'\t{key}')
    
print('\nkeys in the "LCF" dictionary')
for key in my_exp.analysis['LCF'].keys():
    print(f'\t{key}')

print('\nkeys of "Fit 1"')
for key in my_exp.analysis['LCF']['Fit 1'].keys():
    print(f'\t{key}')  

In [ ]:
# 2. Visualize basis sets for fitting

# MODIFY: fit_name, emin, emax

fit_name = 'Fit 1'
emin = 29150
emax = 29300


basis_groups = my_exp.analysis['LCF'][fit_name]['basis spectra']

pfcts.plot_XANES(basis_groups, emin, emax, spectra = 'flat', 
                 deriv = False, e0 = None, e0_line = False, 
                 overlay = True, use_legend = False)

### 4c. Perform LCF

    This fitting step allows different paramters to updated and the fit to be performed.  This step can be run multiple times as needed.

#### MODIFY:
    1. fit_name - which fit is being completed 
    2. emin - starting energy range for the LCF
    3. emax - ending enregy range for the LCF
    4. minvals - minimum values that fits can be at (lower limit of component fraction)
    5. maxvals - maximum values that fits can be at (upper limit of component fraction)
    6. array_name - what spectra to fit, 'mu', 'norm', 'flat' are common options
    7. sum_to_one - True = all components fractions must sum to 1, False = component fractions can some to anything
    

In [ ]:
# Fit the sample spectra
fit_name = 'Fit 1'
emin = 29150
emax = 29250
minvals = 0
maxvals = 1
array_name = 'flat'
sum_to_one = True

my_exp.fit_LCF(fit_name, emin, emax, 
               weights = None, minvals = minvals, maxvals = maxvals, 
               arrayname = array_name, sum_to_one = sum_to_one)

#### Inspection tool to show how LCF is updated
    1. Dictionary of fit - experiment.analysis['LCF'][fit_name]
    2. LCF result for a single spectra

In [ ]:
# 1. inspection of the updated fit 

print('keys of "Fit 1"')
for key in my_exp.analysis['LCF']['Fit 1'].keys():
    print(f'\t{key}') 

In [ ]:
# 2. LCF results for the first scan 
my_exp.analysis['LCF']['Fit 1']['Results'][scan_name]

### 4d. Generate a Fit Report

    This step compiles all the LCF results for each spectr into a easy to read dataframe into "Fit Summary"
    
#### Modify:
    
    1. fit_name - name of fit to generate report on

In [ ]:
# Generate fit report for all spectra
fit_name = 'Fit 1'

my_exp.lcf_report('Fit 1')

#### Inspection tool to show how LCF is updated
    1. Dictionary of fit - experiment.analysis['LCF'][fit_name]
    2. Fit Summary results

In [ ]:
# 1. inspection of the updated fit contents

print('keys of "Fit 1"')
for key in my_exp.analysis['LCF']['Fit 1'].keys():
    print(f'\t{key}')

In [ ]:
# 2. Contents of the Fit Summary
my_exp.analysis['LCF']['Fit 1']['Fit Summary']

### 4e. Visualize Fit and Errors with one Process Parameter

#### Modify:
    1. fit_name
    2. process_param - name of parameter from a process stream. Common option is temperature (from LV:'TR1 T [°C]')

In [ ]:
# Visualize Fit with process parameter

fit_name = 'Fit 1'
process_param = 'TR1 T [°C]'

my_exp.plot_LCF_results(fit_name, process_parameter = process_param)

### 4f. Visulaize Single Fit

#### Modify:
    1. fit_name
    2. emin/emax - Energy range to plot over 
    3. ymin/ymax - Adjsutd scaling as needed

In [ ]:
# Visualize individual fits

fit_name = 'Fit 1'
emin = 29150
emax = 29250
ymin = -0.05
ymax = 1.6


interact(my_exp.plot_LCF, spectra = my_exp.spectra.keys(),
        fit_name = fixed(fit_name),
        emin = fixed(emin), emax = fixed(emax), 
        ymin = fixed(ymin), ymax = fixed(ymax))



## Step 5. Save Experiment with 'pickle' 
    Pickle - A method to save the experimental class and reopen it elsewhere.

In [ ]:
pickle_out = open(pickle_path,"wb")
pickle.dump(my_exp, pickle_out)
pickle_out.close()